In [8]:
import pandas as pd

# データフレームの作成
data = {
    'Points limit': [500, 1000, 10000, 500, 1000, 10000],
    'Proposed (5%)': [1, 9, 0.083, 25, 33, 0.106],
    'ORB': [2, 10, 0.361, 26, 34, 0.576],
    # 他の列も同様に追加
}
df = pd.DataFrame(data)

# LaTeX形式で基本的なテーブルを出力
latex_table = df.to_latex(index=False)
print(latex_table)

\begin{tabular}{rrr}
\toprule
 Points limit &  Proposed (5\%) &    ORB \\
\midrule
          500 &          1.000 &  2.000 \\
         1000 &          9.000 & 10.000 \\
        10000 &          0.083 &  0.361 \\
          500 &         25.000 & 26.000 \\
         1000 &         33.000 & 34.000 \\
        10000 &          0.106 &  0.576 \\
\bottomrule
\end{tabular}



In [9]:
import csv
import os
import numpy as np
import matplotlib.pyplot as plt
import math

COMPARE_DESCRIPTORS = ["Proposed", "orb", "sphorb", "sift", "spoint", "alike", ]
ABLATION_DESCRIPTORS = ["Proposed","Proposed01", "Proposed1", "Proposed10", "Proposed_un", "Ltspoint", "tspoint", "Ftspoint"] 
NOT_T_DESCRIPTORS = ["spoint", "tspoint", "sphorb", "Ltspoint", "Proposed01", "Proposed03", "Proposed05", "Proposed1", "Proposed3", "Proposed", "Proposed10", "Proposed20", "Proposed_un", "Ftspoint"]
METHODS = ["", "t"]
PARAMS = ["R", "T"]
PARAMS_DICT = {"R": "Rotation", "T": "Translation"}
ALL_LOCS = ["Classroom", "Room", "Realistic", "Interior1", "Interior2", "Urban1", "Urban2", "Urban3", "Urban4"]
INDOORS = ["Classroom", "Room", "Realistic", "Interior1", "Interior2"]
OUTDOORS = ["Urban1", "Urban2", "Urban3", "Urban4"]
POSES = ["pose1", "pose2", "pose3", "pose4", "pose5"]
TIME_CATEGORYS = ["FP", "MC", "PE"]
DESCRIPTORS_DICT = {"Proposed1": "Proposed (1%)",
                    "Proposed01": "Proposed (0.1%)",
                    "Proposed03": "Proposed (0.3%)",
                    "Proposed05": "Proposed (0.5%)",
                    "Proposed3": "Proposed (3%)",
                    "Proposed": "Proposed (5%)",
                    "Proposed10": "Proposed (10%)",
                    "Proposed20": "Proposed (20%)",
                    "Proposed_un": "MNN",
                    "orb": "ORB", 
                    "sphorb": "SPHORB",
                    "sift": "SIFT",
                    "alike": "ALIKE",
                    "spoint": "SPoint",
                    "tspoint": "KNN+L",
                    "Ltspoint": "Proposed (5%)+L",
                    "Ftspoint": "FLANN"
                    }
METHODS_DICT = {"t": "T", "":""}

In [10]:
def read_csv_data(file_path):
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        data = [float(row[0]) for row in reader]
    return data

In [11]:
def make_latex_table_time():
    for i, param in enumerate(PARAMS):
        for j, loc in enumerate(["indoor", "outdoor"]):
            if loc == "indoor":
                locs = INDOORS
            else:
                locs = OUTDOORS
            time_data = {'Points limit': ["ALL", 500, 1000, 10000]}
            time_compare_df = pd.DataFrame(time_data)
            time_ablation_df = pd.DataFrame(time_data)
            for tgt in ["compare", "ablation"]:
                if tgt == "compare":
                    descriptors = COMPARE_DESCRIPTORS
                    time_df = time_compare_df
                else:
                    descriptors = ABLATION_DESCRIPTORS
                    time_df = time_ablation_df
                for descriptor in descriptors:
                    for method in METHODS:
                        if descriptor in NOT_T_DESCRIPTORS and method == "t":
                            continue
                        method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                        time_df[method_tmp] = [0, 0, 0, 0]
        
            for idx_points, num_points in enumerate([500, 1000, 10000]):
                base_path = f"results/FP_{num_points}/values"
                value_dict = {}
                for tgt in ["compare", "ablation"]:
                    if tgt == "compare":
                        descriptors = COMPARE_DESCRIPTORS
                        time_df = time_compare_df
                    else:
                        descriptors = ABLATION_DESCRIPTORS
                        time_df = time_ablation_df
                    for time_category in TIME_CATEGORYS:
                        #print(time_category)
                        for descriptor in descriptors:
                            for method in METHODS:
                                if descriptor in NOT_T_DESCRIPTORS and method == "t":
                                    continue
                                method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                                all_time_data = []
                                for scene in ALL_LOCS:
                                    file_path = f"{base_path}/{scene}_{method}{descriptor}_5PA_GSM_wRT/TIMES_{time_category}.csv"
                                    time_data = read_csv_data(file_path)
                                    all_time_data.extend(time_data)
                                if time_category == "FP":
                                    time_df.loc[0, method_tmp] = np.mean(all_time_data)
                                elif time_category == "MC":
                                    time_df.loc[idx_points+1, method_tmp] = np.mean(all_time_data)
        
            time_latex_table = time_compare_df.to_latex(index=False,  float_format="{:.3f}".format)
            print(time_latex_table)
            time_latex_table = time_ablation_df.to_latex(index=False, float_format="{:.3f}".format)
            print(time_latex_table)

make_latex_table_time()

\begin{tabular}{lrrrrrrrrr}
\toprule
Points limit &  Proposed (5\%) &   ORB &  TORB &  SPHORB &  SIFT &  TSIFT &  SPoint &  ALIKE &  TALIKE \\
\midrule
         ALL &          0.483 & 0.194 & 0.218 &   0.143 & 0.223 &  0.668 &   0.236 &  0.186 &   0.331 \\
         500 &          0.008 & 0.011 & 0.013 &   0.001 & 0.010 &  0.011 &   0.011 &  0.010 &   0.011 \\
        1000 &          0.013 & 0.010 & 0.013 &   0.002 & 0.009 &  0.010 &   0.011 &  0.010 &   0.011 \\
       10000 &          0.015 & 0.084 & 0.058 &   0.105 & 0.012 &  0.011 &   0.014 &  0.012 &   0.012 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrrrrr}
\toprule
Points limit &  Proposed (5\%) &  Proposed (0.1\%) &  Proposed (1\%) &  Proposed (10\%) &   MNN &  Proposed (5\%)+L &  KNN+L &  FLANN \\
\midrule
         ALL &          0.483 &            0.495 &          0.500 &           0.496 & 0.481 &            0.483 &  0.483 &  0.481 \\
         500 &          0.008 &            0.013 &          0.013 &           0.011 & 0

In [12]:
def make_latex_table_MAEMSE():
    for i, param in enumerate(PARAMS):
        for j, loc in enumerate(["indoor", "outdoor"]):
            if loc == "indoor":
                locs = INDOORS
            else:
                locs = OUTDOORS
            print(param, loc)
            maemse_data = {'Points limit': [500, 1000, 10000, 500, 1000, 10000]}
            maemse_compare_df = pd.DataFrame(maemse_data)
            maemse_ablation_df = pd.DataFrame(maemse_data)
            for tgt in ["compare", "ablation"]:
                if tgt == "compare":
                    descriptors = COMPARE_DESCRIPTORS
                    maemse_df = maemse_compare_df
                else:
                    descriptors = ABLATION_DESCRIPTORS
                    maemse_df = maemse_ablation_df
                for descriptor in descriptors:
                    for method in METHODS:
                        if descriptor in NOT_T_DESCRIPTORS and method == "t":
                            continue
                        method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                        maemse_df[method_tmp] = [0, 0, 0, 0, 0, 0]
        
            for idx_points, num_points in enumerate([500, 1000, 10000]):
                base_path = f"results/FP_{num_points}/values"
                value_dict = {}
                for tgt in ["compare", "ablation"]:
                    if tgt == "compare":
                        descriptors = COMPARE_DESCRIPTORS
                        maemse_df = maemse_compare_df
                    else:
                        descriptors = ABLATION_DESCRIPTORS
                        maemse_df = maemse_ablation_df
                    for descriptor in descriptors:
                        for method in METHODS:
                            if descriptor in NOT_T_DESCRIPTORS and method == "t":
                                continue
                            all_error_data = []
                            for scene in locs:
                                file_path = f"{base_path}/{scene}_{method}{descriptor}_5PA_GSM_wRT/{param}_ERRORS.csv"
                                error_data = read_csv_data(file_path)
                                all_error_data.extend(error_data)
                            method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                            maemse_df.loc[idx_points, method_tmp] = np.mean(all_error_data)
                            maemse_df.loc[idx_points+3, method_tmp] = np.mean(np.array(all_error_data) ** 2)
        
            maemse_latex_table = maemse_compare_df.to_latex(index=False,  float_format="{:.3f}".format)
            print(maemse_latex_table)
            maemse_latex_table = maemse_ablation_df.to_latex(index=False, float_format="{:.3f}".format)
            print(maemse_latex_table)

make_latex_table_MAEMSE()

R indoor
\begin{tabular}{rrrrrrrrrr}
\toprule
 Points limit &  Proposed (5\%) &   ORB &  TORB &  SPHORB &  SIFT &  TSIFT &  SPoint &  ALIKE &  TALIKE \\
\midrule
          500 &          0.083 & 0.914 & 0.667 &   0.786 & 0.544 &  0.411 &   0.758 &  2.975 &   2.782 \\
         1000 &          0.092 & 0.832 & 0.559 &   0.526 & 0.281 &  0.229 &   0.734 &  2.923 &   2.681 \\
        10000 &          0.083 & 0.361 & 0.184 &   0.246 & 0.129 &  0.125 &   0.758 &  2.847 &   2.677 \\
          500 &          0.078 & 1.860 & 1.218 &   1.685 & 0.932 &  0.868 &   2.176 &  9.305 &   8.650 \\
         1000 &          0.123 & 1.811 & 1.144 &   1.082 & 0.378 &  0.495 &   2.057 &  9.129 &   8.300 \\
        10000 &          0.106 & 0.576 & 0.276 &   0.460 & 0.105 &  0.214 &   2.099 &  8.872 &   8.286 \\
\bottomrule
\end{tabular}

\begin{tabular}{rrrrrrrrr}
\toprule
 Points limit &  Proposed (5\%) &  Proposed (0.1\%) &  Proposed (1\%) &  Proposed (10\%) &   MNN &  Proposed (5\%)+L &  KNN+L &  FLANN \\
\

In [15]:
def make_latex_table_MAEMSE_farm():
    for i, param in enumerate(PARAMS):
        print(param)
        maemse_data = {'Points limit': [500, 1000, 10000, 500, 1000, 10000]}
        maemse_compare_df = pd.DataFrame(maemse_data)
        maemse_ablation_df = pd.DataFrame(maemse_data)
        for tgt in ["compare", "ablation"]:
            if tgt == "compare":
                descriptors = COMPARE_DESCRIPTORS
                maemse_df = maemse_compare_df
            else:
                descriptors = ABLATION_DESCRIPTORS
                maemse_df = maemse_ablation_df
            for descriptor in descriptors:
                for method in METHODS:
                    if descriptor in NOT_T_DESCRIPTORS and method == "t":
                        continue
                    method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                    maemse_df[method_tmp] = [0, 0, 0, 0, 0, 0]
    
        for idx_points, num_points in enumerate([500, 1000, 10000]):
            base_path = f"results/FP_{num_points}/values"
            value_dict = {}
            for tgt in ["compare", "ablation"]:
                if tgt == "compare":
                    descriptors = COMPARE_DESCRIPTORS
                    maemse_df = maemse_compare_df
                else:
                    descriptors = ABLATION_DESCRIPTORS
                    maemse_df = maemse_ablation_df
                for descriptor in descriptors:
                    for method in METHODS:
                        if descriptor in NOT_T_DESCRIPTORS and method == "t":
                            continue
                        all_error_data = []
                        for scene in POSES:
                            file_path = f"{base_path}/{scene}_{method}{descriptor}_5PA_GSM_wRT/{param}_ERRORS.csv"
                            error_data = read_csv_data(file_path)
                            all_error_data.extend(error_data)
                        method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                        maemse_df.loc[idx_points, method_tmp] = np.mean(all_error_data)
                        maemse_df.loc[idx_points+3, method_tmp] = np.mean(np.array(all_error_data) ** 2)
    
        maemse_latex_table = maemse_compare_df.to_latex(index=False,  float_format="{:.4f}".format)
        print(maemse_latex_table)
        maemse_latex_table = maemse_ablation_df.to_latex(index=False, float_format="{:.4f}".format)
        print(maemse_latex_table)

make_latex_table_MAEMSE_farm()

R
\begin{tabular}{rrrrrrrrrr}
\toprule
 Points limit &  Proposed (5\%) &    ORB &   TORB &  SPHORB &   SIFT &  TSIFT &  SPoint &  ALIKE &  TALIKE \\
\midrule
          500 &         0.0447 & 0.0655 & 0.0643 &     NaN & 0.0488 & 0.0445 &  0.0448 & 0.0571 &  0.0692 \\
         1000 &         0.0451 & 0.0490 & 0.0474 &     NaN & 0.0439 & 0.0433 &  0.0445 & 0.0600 &  0.0638 \\
        10000 &         0.0451 & 0.0544 & 0.0521 &     NaN & 0.0434 & 0.0432 &  0.0440 & 0.0579 &  0.0743 \\
          500 &         0.0060 & 0.0477 & 0.0449 &     NaN & 0.0069 & 0.0061 &  0.0059 & 0.0102 &  0.0230 \\
         1000 &         0.0060 & 0.0066 & 0.0064 &     NaN & 0.0063 & 0.0060 &  0.0059 & 0.0146 &  0.0147 \\
        10000 &         0.0060 & 0.0075 & 0.0073 &     NaN & 0.0063 & 0.0060 &  0.0059 & 0.0084 &  0.0527 \\
\bottomrule
\end{tabular}

\begin{tabular}{rrrrrrrrr}
\toprule
 Points limit &  Proposed (5\%) &  Proposed (0.1\%) &  Proposed (1\%) &  Proposed (10\%) &    MNN &  Proposed (5\%)+L &  KNN+